# Импорт либ

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings
for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))
warnings.simplefilter("ignore")
%matplotlib inline
sns.set(style="darkgrid")

# Загрузка & Анализ данных

In [ ]:
df = pd.read_csv("/kaggle/input/advanced-dls-spring-2021/train.csv")
df.shape

In [ ]:
df.head()

In [ ]:
df.columns

Ну как там с пропусками?

In [ ]:
df.isna().sum()

Ну как там с дубликатами?

In [ ]:
df.duplicated().sum()

Ну как там с дубликатами без целевой переменной? Удалим те, у которых таргет 0, так как их скорее всего ошибочно не удалили из базы(поскольку дубликаты с таргетом 1 появились в бд позже)

In [ ]:
y = df["Churn"]
X = df.drop(columns=["Churn"])
duplicates = X[X.duplicated(keep=False)].merge(y[X.duplicated(keep=False)],
                                               left_index=True,
                                               right_index=True).sort_values(by=["TotalSpent", "Churn"]).index[::2]
X.drop(index=duplicates, inplace=True)
y.drop(index=duplicates, inplace=True)

Ну как там с распределением таргета?

In [ ]:
y.nunique()

In [ ]:
sns.displot(y)

Ну как там с признаками?

In [ ]:
X.describe()

In [ ]:
X.dtypes

Разделим их на числовые и категориальные

In [ ]:
numeric = list(X.select_dtypes([np.number]).columns)
categorical = list(X.dtypes[X.dtypes == "object"].index)
print("Численные фичи:\n\t", "\n\t".join(numeric),
      "\n\nКатегориальные фичи:\n\t", "\n\t".join(categorical), sep="")

Ну как там с числовыми признаками?

In [ ]:
for feature in numeric:
    print(feature, X[feature].nunique())

IsSeniorCitizen выглядит как категориальный

In [ ]:
categorical.append("IsSeniorCitizen")
numeric.remove("IsSeniorCitizen")
X.IsSeniorCitizen.unique()

Ну как там с категориальными фичами?

In [ ]:
for feature in categorical:
    print(feature, X[feature].nunique())

TotalSpent похож на числовой признак

In [ ]:
X.TotalSpent.value_counts()

Найдём, почему это не числовой признак

In [ ]:
defect = []
for value in X.TotalSpent.value_counts().index:
    try:
        float(value)
    except:
        defect.append(value)
defect

In [ ]:
X.TotalSpent.value_counts()[" "]

Заметим, что пробелы стоят там, где срок равен 0, заменим их на 0 и добавим логарифм от этой фичи

In [ ]:
X["TotalSpent"][X["TotalSpent"] == " "] = "0"
X["TotalSpent"] = X["TotalSpent"].astype(float)
X["TotalSpentLog"] = np.log(X["TotalSpent"] + 1)
numeric.append("TotalSpentLog")
numeric.append("TotalSpent")
categorical.remove("TotalSpent")

Ну как там с распределениями категориальных признаков?

In [ ]:
for i in range(4):
    fig, axs = plt.subplots(figsize=(16, 5), ncols=4)
    for i, feature in enumerate(X[categorical[i * 4:(i + 1) * 4]]):
        axs[i].hist(X[feature])
        axs[i].set_xlabel(feature)
        axs[i].set_ylabel("count")
    plt.tight_layout()

Заметим повторяющееся значение  No internet servie, в разных признаках. Добавим их в отдельную категорию

In [ ]:
print(len(X))
without_internet = []
for feature in categorical:
    if "No internet service" in X[feature].unique():
        without_internet.append(feature)
        print(f"{feature}: ", sum((X[feature] == "No internet service") == (X["HasInternetService"] == "No")))
for feature in without_internet:
    categorical.remove(feature)

Ну как там с корреляциями?

In [ ]:
correlations = X[numeric].corrwith(y).sort_values(ascending=False)
plt.figure(figsize=(14, 4))
sns.barplot(y=correlations.index, x=correlations)

Ну как там с зависимостью таргета от численных фичей?

In [ ]:
fig, axs = plt.subplots(figsize=(16, 5), ncols=4)
for i, feature in enumerate(numeric):
    axs[i].scatter(X[feature], y, alpha=0.2)
    axs[i].set_xlabel(feature)
    axs[i].set_ylabel("Churn")
plt.tight_layout()

# Линейный пайплайн

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [ ]:
from sklearn.linear_model import LogisticRegression
column_transformer = ColumnTransformer([
    ("ohe", OneHotEncoder(handle_unknown="ignore",
                         drop="first"), categorical),
    ("ohe_drop", OneHotEncoder(handle_unknown="ignore",
                               drop=6*["No internet service"]), without_internet),
    ("scaling", StandardScaler(), numeric)
])
logreg_pipeline = Pipeline(steps=[
    ("ohe_and_scaling", column_transformer),
    ("classification", LogisticRegression(penalty="l2", max_iter=10000))
])
params = {"classification__C": [0.01, 0.1, 1, 10, 100],
          "classification__class_weight": [None, "balanced"],
          "classification__solver": ["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga"]
         }
search_logreg = GridSearchCV(logreg_pipeline, params, n_jobs=-1, cv=3, refit=True, scoring="roc_auc", verbose=0)
search_logreg.fit(X_train, y_train)
logreg_pipeline.set_params(**search_logreg.best_params_)
logreg_pipeline.fit(X_train, y_train)
roc_auc_score(y_test, logreg_pipeline.predict_proba(X_test)[:, 1])

Получилось на тесте около 0.847

# Пайплайн SVM

Решил не добавлять сюда, потому что с позором

Получилось около 0.843

# Лесной пайплайн

Будем кодировать категориальные фичи OHE, так как скор почему-то получился лучше LE

In [ ]:
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
forest_pipeline = Pipeline(steps=[
    ("scaling", column_transformer),
    ("classification", RandomForestClassifier())
])
params = {"classification__n_estimators": [50, 75, 100, 150],
          "classification__criterion": ["gini", "log_loss", "entropy"],
          "classification__max_depth": range(4, 9)}
search_forest = GridSearchCV(forest_pipeline, params, n_jobs=-1, cv=3, refit=True,
                                scoring="roc_auc", verbose=0)
search_forest.fit(X_train, y_train)
forest_pipeline.set_params(**search_forest.best_params_)
forest_pipeline.fit(X_train, y_train)
roc_auc_score(y_test, forest_pipeline.predict_proba(X_test)[:,1])

Получилось на тесте около 0.848

# Пайплайн бустинга

In [ ]:
categorical.extend(without_internet)

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(cat_features=categorical)
model.fit(X_train, y_train, verbose=False)
roc_auc_score(y_test, model.predict_proba(X_test)[:,1])

In [ ]:
fix_params = {"cat_features": categorical,
              "loss_function": "Logloss",
              "custom_metric": "AUC:hints=skip_train~false;type=Classic",
              "eval_metric": "AUC:hints=skip_train~false;type=Classic",
              "verbose": False
}

Без оптуны ну очень долго подбиралось, пришлось подрубить её

In [ ]:
import optuna
from optuna.samplers import TPESampler
def objective(trial):
    model = CatBoostClassifier(
        **fix_params,
        score_function=trial.suggest_categorical("score_function", ["Cosine", "L2"]),
        depth=trial.suggest_int("depth", 4, 6),
        auto_class_weights=trial.suggest_categorical("auto_class_weights", [None, "Balanced", "SqrtBalanced"]),
        grow_policy=trial.suggest_categorical("grow_policy", ["Depthwise", "SymmetricTree", "Lossguide"]),
        iterations=trial.suggest_int("iterations", 100, 1000),
        learning_rate=trial.suggest_float("learning_rate", .03, .9),
        model_shrink_rate=trial.suggest_float("model_shrink_rate", .001, .9),
        model_shrink_mode=trial.suggest_categorical("model_shrink_mode", ["Constant", "Decreasing"]),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1, 1000, log=True),
        leaf_estimation_method=trial.suggest_categorical("leaf_estimation_method", ["Newton", "Gradient"]),
        leaf_estimation_iterations=trial.suggest_int("leaf_estimation_iterations", 1, 10),
        leaf_estimation_backtracking=trial.suggest_categorical("leaf_estimation_backtracking", ["No", "AnyImprovement"]),
    )
    model.fit(X_train, y_train)
    return roc_auc_score(y_test, model.predict_proba(X_test)[:,1])
optuna.logging.set_verbosity(optuna.logging.WARNING)
sampler = TPESampler(seed=0)
study = optuna.create_study(study_name="catboost", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=200)
trial = study.best_trial
trial.value

Все параметры после этого вручную в словарь, чтобы закомментить долгий подбор параметров и скор быстрее пришёл, ну и да, iterations пришлось после этого дополнительно рандомить вручную для лучшего скора(там было около 507)

In [ ]:
trial.params

In [ ]:
res_params = {
    "score_function": "Cosine",
    "depth": 5,
    "auto_class_weights": "Balanced",
    "grow_policy": "Lossguide",
    "iterations": 937,
    "learning_rate": 0.12330483781630441,
    "model_shrink_rate": 0.567999516152233,
    "model_shrink_mode": "Decreasing",
    "l2_leaf_reg": 392.6719565225887,
    "leaf_estimation_method": "Newton",
    "leaf_estimation_iterations": 1,
    "leaf_estimation_backtracking": "No"
}
model = CatBoostClassifier(
    **fix_params,
    **res_params
)
model.fit(X, y)

Загрузим и предобработаем тестовые данные

In [ ]:
test = pd.read_csv("/kaggle/input/advanced-dls-spring-2021/test.csv")
test["TotalSpent"][test["TotalSpent"] == " "] = "0"
test["TotalSpent"] = test["TotalSpent"].astype(float)
test["TotalSpentLog"] = np.log(test["TotalSpent"] + 1)

Получим вероятности бустинга...

In [ ]:
pred = model.predict_proba(test)

... и загрузим их в csv

In [ ]:
sub = pd.read_csv("/kaggle/input/advanced-dls-spring-2021/submission.csv")
sub["Churn"] = pred[:, 1]
sub.to_csv("submission.csv", index=False)
sub = pd.read_csv("/kaggle/working/submission.csv")
sub

Итоговый скор: 0.85486

Статистика сабмитов:
    logreg: 12
    svc: 9
    randomforest: 10
    catboost: 61